In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import h5py
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

In [ ]:
val_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/train_set_class_nn.pickle')
train_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/val_set_class_nn.pickle')


In [ ]:
model_save_dir ='models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-cross-val1'
os.makedirs(model_save_dir,exist_ok=True)

In [ ]:
image_shape = (128,128,1)
input_tensor = Input(image_shape)
inceptionresnetv2 = InceptionResNetV2(input_tensor=input_tensor, weights=None, include_top=True, classes=2)

In [ ]:
#batch size
train_batch_size = 16
val_batch_size = 16
nepochs = 50
#model optimizer
optimizer = Adam(beta_1=0.99,beta_2=0.99,lr=1.0e-4,decay=1.0e-6)
#loss
loss= 'categorical_crossentropy'
#create checkpoint callback to save model on epochs that have best validation accuracy
checkpoint = ModelCheckpoint(model_save_dir + '/weights-improvement-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
inceptionresnetv2.compile(loss=loss, optimizer=optimizer, metrics = ['categorical_accuracy'])      

In [ ]:
def get_xy_data(fmatrix):
    x_input = []
    y_output = []
    
    for index, row in fmatrix.iterrows():
        print('loading sequence : {}'.format(row['image_path']))
        img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
        img = cv2.equalizeHist(img)
#         clahe = cv2.createCLAHE()
#         img = clahe.apply(img)
        img = cv2.resize(img,(128,128))
        img = img/255.0
        x_input.append(img.reshape(img.shape + (1,)))
        y_output.append(list(row['lesion_state_one_hot']))
        
    return np.array(x_input), np.array(y_output)    

In [ ]:
#Train Set
print('***build training set***')
x_train, y_train = get_xy_data(train_set)

#Test Set
print('***build val set***')
x_val, y_val = get_xy_data(val_set)

In [ ]:
#Training Set Distribution
print('training set shape-->')
print(x_train.shape)
print('validation set shape-->')
print(x_val.shape)

In [ ]:
#steps per epoch
train_steps_per_epoch = int(np.ceil(len(x_train)/train_batch_size))
val_steps_per_epoch = int(np.ceil(len(x_train)/val_batch_size))

In [ ]:
#Generator for keras model
seed = 9

data_gen_args = dict(width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=10,
                    horizontal_flip=True,
                    fill_mode='nearest')

# train_image_datagen = ImageDataGenerator(**data_gen_args)   
train_image_datagen = ImageDataGenerator()   
val_image_datagen = ImageDataGenerator()   

training_set_generator = train_image_datagen.flow(x_train, y_train, batch_size=train_batch_size, shuffle=True,seed=seed)
validation_set_generator = val_image_datagen.flow(x_val, y_val, batch_size=val_batch_size, shuffle=True, seed=seed)


In [ ]:
history = inceptionresnetv2.fit_generator(training_set_generator, validation_data = validation_set_generator, steps_per_epoch=train_steps_per_epoch, validation_steps=val_steps_per_epoch, epochs = nepochs, callbacks=callbacks_list, verbose=True)

In [ ]:
with h5py.File(model_save_dir+'_history.hdf5') as hfile:
    for keyi in history.history:
        hfile.create_dataset(keyi, data=history.history[keyi])